Telecom X - Análise de Evasão de Clientes
Você foi contratado como assistente de análise de dados na Telecom X e fará parte do projeto "Churn de Clientes". A empresa enfrenta um alto índice de cancelamentos e precisa entender os fatores que levam à perda de clientes.

Seu desafio será coletar, tratar e analisar os dados, utilizando Python e suas principais bibliotecas para extrair insights valiosos. A partir da sua análise, os demais colegas da  equipe de Data Science poderá avançar para modelos preditivos e desenvolver estratégias para reduzir a evasão.

O que você vai praticar:

✅ Importar e manipular dados de uma API de forma eficiente.

✅ Aplicar os conceitos de ETL (Extração, Transformação e Carga) na preparação dos dados.

✅ Criar visualizações de dados estratégicas para identificar padrões e tendências.

✅ Realizar uma Análise Exploratória de Dados (EDA) e gerar um relatório com insights relevantes.

Dicionário de dados

**customerID:** número de identificação único de cada cliente

**Churn:** se o cliente deixou ou não a empresa

**gender:** gênero (masculino e feminino)

**SeniorCitizen:** informação sobre um cliente ter ou não idade igual ou maior que 65 anos

**Partner:** se o cliente possui ou não um parceiro ou parceira

**Dependents:** se o cliente possui ou não dependentes

**tenure:** meses de contrato do cliente

**PhoneService:** assinatura de serviço telefônico

**MultipleLines:** assisnatura de mais de uma linha de telefone

**InternetService:** assinatura de um provedor internet

**OnlineSecurity:** assinatura adicional de segurança online

**OnlineBackup:** assinatura adicional de backup online

**DeviceProtection:** assinatura adicional de proteção no dispositivo

**TechSupport:** assinatura adicional de suporte técnico, menos tempo de espera

**StreamingTV:** assinatura de TV a cabo

**StreamingMovies:** assinatura de streaming de filmes

**Contract:** tipo de contrato

**PaperlessBilling:** se o cliente prefere receber online a fatura

**PaymentMethod:** forma de pagamento

**Charges.Monthly:** total de todos os serviços do cliente por mês

**Charges.Total:** total gasto pelo cliente


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go


# **Extração de dados do GitHub** (**E**TL)

In [ ]:
url = 'https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/refs/heads/main/TelecomX_Data.json'
df = pd.read_json(url)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
#Normalizando o arquivo Json

registros = df.to_dict(orient='records') # convertendo todas as linhas em listas de dicionários:

df_flat = pd.json_normalize(registros)

df_flat.head(10)

Como a função que normaliza arquivos json só aceita dicionários ou listas de dicionarios, criamos uma variavel registros que é a conversão do o nosso dataframe em uma lista dicionarios orientado pelas linhas do Df, assim podemos aplicar a função de normalização em todo o dataframe direto sem ter que normalizar coluna a coluna.

# **Transformação dos dados** (E**T**L)

### **Conhecendo nosso dataset!**

In [ ]:
type(df_flat) # Confirmando a criação do nosso Dataframe

In [ ]:
df_flat.shape #Verificando o número de linhas e colunas

In [ ]:
df_flat.columns #Verificando o nome das colunas

Podemos ver que agora temos um Dataframe com todas colunas condizentes com as variáveis passadas para a gente no inicio do projeto dentro do dicionário. Para uma análise de evasão de clientes, as colunas mais interressantes parecem ser: meses de contratos, gastos mensais, gasto total, tipo de contrato, forma de pagamento, seviços no geral e dentre outros.

In [ ]:
df_flat.info()

Em uma primeira vista, após a normalização, podemos ver que temos um total de 21 colunas e 7267 linhas ou entradas. Dentro das colunas, não temos nenhum valor nulo e podemos confirmar isso posteriormente com alguns métodos.
Algumas variaveis estão com tipagem objeto e talvez seja preciso formatar

### **Verificando inconsistências!**

In [ ]:
#Checando valores ausentes (Null)
missing_counts = df_flat.isnull().sum()
print("Nulos por coluna:\n", missing_counts)


Bom como já haviamos tido uma prévia com o método info, não temos valores ausentes dentro das nossas colunas.

In [ ]:
#Verificando se há linhas duplicadas considerando todas as colunas
num_duplicated = df_flat.duplicated().sum()
print(f"Total de linhas 100% duplicadas: {num_duplicated}")

### **Verificando a tipagem de dados**


In [ ]:
print("Tipos de dados (dtypes):\n")
print(df_flat.dtypes)

Olhando primeiro para as variáveis numéricas, aparetemente a unica que precisamos tratar é a Charges.Total que precisa ser formatada para float64

In [ ]:
# Não podemos converter diretamente uma string para um float. Primeiro precisamos converter a string para um valor numérico
df_flat['account.Charges.Total'] = pd.to_numeric(df_flat['account.Charges.Total'], errors='coerce')

df_flat['account.Charges.Total'].isnull().sum()

Após a conversão da string em numérico ficamos com 11 valores que resultaram NaN que precisam ser tratados.

In [ ]:
# Preenchendo os valores nulos por zero
df_flat['account.Charges.Total'] = df_flat['account.Charges.Total'].fillna(0)
df_flat['account.Charges.Total'] = df_flat['account.Charges.Total'].astype(np.float64)
print(df_flat['account.Charges.Total'].dtype)

In [ ]:
df_flat.info()

Agora todos os dados já estão com os tipos corretos e não temos valores nulos

### **Vamos melhorar os nomes das nossas colunas para ficar mais facil de trabalhar**

In [ ]:
df_flat.columns = df_flat.columns.str.strip().str.lower().str.replace('.','_')

In [ ]:
colunas_traduzidas = {
    'customerid': 'id_cliente',
    'churn': 'evasao',
    'customer_gender': 'genero',
    'customer_seniorcitizen': 'cliente_idoso',
    'customer_partner': 'possui_parceiro',
    'customer_dependents': 'possui_dependentes',
    'customer_tenure': 'tempo_contrato_meses',
    'phone_phoneservice': 'servico_telefonico',
    'phone_multiplelines': 'multiplas_linhas',
    'internet_internetservice': 'tipo_internet',
    'internet_onlinesecurity': 'seguranca_online',
    'internet_onlinebackup': 'backup_online',
    'internet_deviceprotection': 'protecao_dispositivo',
    'internet_techsupport': 'suporte_tecnico',
    'internet_streamingtv': 'streaming_tv',
    'internet_streamingmovies': 'streaming_filmes',
    'account_contract': 'tipo_contrato',
    'account_paperlessbilling': 'fatura_online',
    'account_paymentmethod': 'forma_pagamento',
    'account_charges_monthly': 'gastos_mensais',
    'account_charges_total': 'gastos_totais'
}


df_flat.rename(columns=colunas_traduzidas, inplace=True)

df_flat.head(15)

### **Removendo registros que podem atrapalhar as análises futuras**


Primeiramente, vamos ver quais valores existem dentro de cada coluna do nosso DataFrame

In [ ]:
for i in df_flat.select_dtypes(include='object').columns:
    print(f"{i}: {df_flat[i].unique()}")

Bom, podemos ver que na primeira coluna existem valores vazios e precisamos arrumar isso.

Além disso, é possivel perceber que existem muito valores do tipo booleano (Sim ou não) e seria interressante nos certificamos que todos esses valores seguem um padrão.


In [ ]:
for i in df_flat.select_dtypes(include='number').columns:
    print(f"{i}: {df_flat[i].unique()}")

Olhando agora para valores numéricos ja conseguimos confirmar certos problemas. Primeiramente, a coluna idosos é binária, ou seja, semanticamente representa sim e não assim como valores do tipo objeto. Entretanto é interessante que a gente realize uma padronização dos nossos dados.

Para as demais colunas o unico valor que chama a atenção é que existem 0 meses para o tempo de contrato e isso não necessariamente é um erro, pois existem clientes que podem ter abertos as suas contas recentemente e ainda não contabilizou 1 mês. Contudo, contar esses clientes nos dados podem gerar problemas futuros, principalmente se precisarmos utilizar algum modelo de ML

In [ ]:
#Verificando quais linhas possuem tempo de contrato igual a zero
df_flat.query('tempo_contrato_meses == 0')

Temos então 11 clientes que possuem tempo de contrato igual a 0 e o interressante é que são os mesmo clientes que anteriomente ficaram com o tempo de contrato igual a NaN. Enfim, como estamos querendo analisar a evasão de clientes e não a entrada de novos clientes, eu vou remover essas 11 linhas do Df e assim garantir que isso não cause problemas futuramente.    

In [ ]:
#Removendo os valores
df_flat = df_flat[df_flat['tempo_contrato_meses'] != 0].copy()
df_flat.query('tempo_contrato_meses == 0')

In [ ]:
df_flat.query('gastos_totais == 0')

Vamos agora ver os dados que estão em branco na evasão de clientes.

In [ ]:
df_flat.query("evasao == '' ")

In [ ]:
#Removendo dados em branco
df_flat = df_flat[df_flat['evasao'] != ''].copy()
df_flat.query("evasao == '' ")

In [ ]:
# Padronização do dataframe, trocando string que indicam que um serviço não ocorre para No
colunas_internet = [
    'seguranca_online',
    'backup_online',
    'protecao_dispositivo',
    'suporte_tecnico',
    'streaming_tv',
    'streaming_filmes'
]

for col in colunas_internet:
    if col in df_flat.columns:
        df_flat[col] = df_flat[col].replace({'No internet service': 'No'})

if 'multiplas_linhas' in df_flat.columns:
    df_flat['multiplas_linhas'] = df_flat['multiplas_linhas'].replace({'No phone service': 'No'})

for col in colunas_internet + ['multiplas_linhas']:
    if col in df_flat.columns:
        print(f"{col}: {df_flat[col].unique()}")

In [ ]:
# Padronizando as variveis booleanas

colunas_binarias = [
    'evasao', 'possui_parceiro', 'possui_dependentes', 'servico_telefonico',
    'multiplas_linhas', 'seguranca_online', 'backup_online',
    'protecao_dispositivo', 'suporte_tecnico', 'streaming_tv',
    'streaming_filmes', 'fatura_online'
]

# Converte 'Yes' para 1 e 'No' para 0
df_flat[colunas_binarias] = df_flat[colunas_binarias].replace({'Yes': 1, 'No': 0})

# Converte explicitamente as colunas para o tipo int64
for col in colunas_binarias:
    df_flat[col] = df_flat[col].astype('int64')

# Verifique os tipos de dados após a conversão
print(df_flat[colunas_binarias].dtypes)

In [ ]:
df_flat

Vamos fazer uma rapida verificação do nosso DataFrame limpo e transformado

In [ ]:
for i in df_flat.select_dtypes(include='object').columns:
    print(f"{i}: {df_flat[i].unique()}")

In [ ]:
for i in df_flat.select_dtypes(include='number').columns:
    print(f"{i}: {df_flat[i].unique()}")

In [ ]:
df_flat.info()

In [ ]:
df_clean = df_flat

### **Criando a coluna de contas Diarias**

In [ ]:
df_clean

In [ ]:
df_clean['contas_diarias'] = df_clean['gastos_mensais'] / 30
df_clean['contas_diarias'] = df_clean['contas_diarias'].round(2)
df_clean[['gastos_mensais','contas_diarias']].head(15)

Conseguimos criar uma coluna que informa o valor das contas diárias de cada cliente com base nos gastos mensais

In [ ]:
df_clean.info()

# **Carregamento e análise** ET**L**


In [ ]:
df_clean.describe()

Vamos para as principais ponderações com base na análise descritiva dos dados.

Temos que todos os dados numéricos estão com as mesmas contagens. Pelo fato de estarmos trabalhando com dados binários os valores mínimos e máximos não terão muito significado para gente, assim como os quartis. Porém podemos levantar algumas observações com base na média e o desvio padrão.  

- Primeiramente, a média de **evasão de clientes** da Telecom X é de 26.57% e o desvio padrão de 44.17%

- Os valores de média para os **gastos mensais** são de aproximadamente R\$ 64,80 reais e o desvio padrão de R\$ 30,08 reais.

- Já para os **gastos totais** temos uma média de R\$ 2.283,30 reais e desvio padrão de R\$2.226,77 reais.

- Para as **contas diárias** a média ficou entre R\$ 2,15 reais e um desvio padrão de R$ 1,00 real

- O **tempo de contrato** médio em meses é de 32 e o desvio padrão de 24 meses. Com minimo de 1 mês e máximo de 72 meses

Com isso e os demais dados podemos fazer um resumo do comportamento dos dados e buscar padrões:


✅ Perfil Geral dos Clientes

Clientes evadidos: 26,6%.

Idosos: apenas 16,2% dos clientes. A maioria são jovens então

Com parceiro(a): ~48%.

Com dependentes: ~30%.

⏳ Tempo de Contrato

Média: 32 meses.

25% dos clientes têm menos de 9 meses de contrato.

Isso faz pensar que existe uma alta evasão no início do vínculo. Ainda antes de fechar 1 ano.

☎️ Serviços adicionais

Menos da metade dos clientes usa serviços como:

Segurança online (28%)

Backup online (34%)

Streaming (38%)

Clientes com mais serviços podem ser mais fiéis. Isso porque dependem da Telecom X para diversas necessidades e trocar esses serviços todos de uma mês é algo mais complicado.

💰 Gastos

Mensais: média de R$ 64,80.

Totais: variam muito, até R$ 8.684.

Contas diárias: média de R$ 2,15 — útil para normalizar análise de gastos.

🧾 Fatura Online

59% preferem fatura online — Confirma o fato de grande parte do publico tem um perfil mais jovem.

## **Gráficos para visualizar a distribuição de evasão dos clientes**

In [ ]:
evasao_clientes = df_clean.evasao.value_counts()
evasao_clientes_norm = df_clean.evasao.value_counts(normalize=True).round(4)*100

In [ ]:
# Adicionando as cores
VERMELHO_1,	VERMELHO_2,	VERMELHO_3 = "#e23155",	"#cc2c4e", "#b32742"
AZUL_1,	AZUL_2,	AZUL_3 = "#203f75",	"#1c3867", "#19325b"
BRANCO,	CINZA_1, CINZA_2, CINZA_3, CINZA_4,	CINZA_5 = "#ffffff", "#ebebeb",	"#d9d9d9", "#cccccc", "#555655", "#231f20"
AQUA_1,	AQUA_2,	AQUA_3 = "#addcd4",	"#9fccc5", "#96bfb9"

### **Distribuição das evasões**

In [ ]:
# Observando a distribuição da evasão de clientes

labels = ['Clientes mantidos', 'Clientes perdidos']

fig, axes = plt.subplots(1, 2, figsize=(13, 8))

# GRÁFICO DE BARRAS
axes[0].bar(labels, evasao_clientes, color=[AQUA_2,VERMELHO_1])
axes[0].set_facecolor(CINZA_1)

# Anotações nos topos das barras
for i, v in enumerate(evasao_clientes):
    axes[0].text(i, v + 1, str(v), ha='center', va='bottom', color=AZUL_1, fontsize=12)

axes[0].set_title('Distribuição em quantidade dos clientes perdidos pela Telecom X',
                  fontsize=14, color=CINZA_5)
axes[0].tick_params(axis='x', labelsize=12, colors=AZUL_1)
axes[0].set_yticks([])
axes[0].set_xlabel('', fontsize=12, color=AZUL_1)


for spine in ['top', 'right', 'left']:
    axes[0].spines[spine].set_visible(False)

axes[0].grid(False)

# GRÁFICO DE PIZZA

axes[1].pie(evasao_clientes_norm, autopct='%1.1f%%', startangle=90, colors=[AQUA_2,VERMELHO_1])
axes[1].set_title('Proporção de Evasão de Clientes', fontsize=16, color=CINZA_5)
axes[1].axis('equal')

plt.tight_layout()
plt.show()

### **Avaliando valores numéricos junto a evasão de clientes**

In [ ]:
# Gráfico 1: Boxplot da evasão vs tempo de contrato
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Boxplot
sns.boxplot(ax=axes[0], x='evasao', y='tempo_contrato_meses', data=df_clean, palette=[AQUA_3, VERMELHO_1])
axes[0].set_facecolor(CINZA_1)
axes[0].set_title('Tempo de Contrato vs Evasão', loc = 'left' , fontsize=14)
axes[0].set_xlabel('Evasão (0 = Não, 1 = Sim)', fontsize=12)
axes[0].set_ylabel('Tempo de Contrato (meses)', fontsize=12)
axes[0].grid(True)

for spine in ['top', 'right']:
    axes[0].spines[spine].set_visible(False)

# Gráfico 2: Histogramas sobrepostos com KDE
sns.histplot(ax=axes[1], data=df_clean[df_clean['evasao'] == 0], x='tempo_contrato_meses', color=AQUA_3, label='Clientes mantidos', kde=True, bins=30)
sns.histplot(ax=axes[1], data=df_clean[df_clean['evasao'] == 1], x='tempo_contrato_meses', color=VERMELHO_1, label='Clientes perdidos', kde=True, bins=30)

axes[1].set_facecolor(CINZA_1)
axes[1].set_title('Distribuição do Tempo de Contrato por Evasão', loc='left', fontsize=14)
axes[1].set_xlabel('Tempo de Contrato (meses)', fontsize=12)
axes[1].set_ylabel('Frequência', fontsize=12)

for spine in ['top', 'right']:
    axes[1].spines[spine].set_visible(False)

axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Criando uma cópia do dataframe com evasao como string
df_plot = df_flat.copy()
df_plot['evasao'] = df_plot['evasao'].map({0: 'Clientes mantidos', 1: 'Clientes perdidos'})

variaveis = ['gastos_mensais', 'gastos_totais', 'contas_diarias']
titulos = ['Gastos Mensais', 'Gastos Totais', 'Contas Diárias']

# Subplots
fig = sp.make_subplots(rows=1, cols=3, subplot_titles=titulos)

# Mapeamento de cor
mapa_cores = {
    'Clientes mantidos': AQUA_3,
    'Clientes perdidos': VERMELHO_1
}

# Adiciona cada scatter
for i, var in enumerate(variaveis):
    scatter = px.scatter(
        df_plot,
        x='tempo_contrato_meses',
        y=var,
        color='evasao',
        color_discrete_map=mapa_cores,
        opacity=0.5,
        labels={
            'evasao': 'Evasão',
            'tempo_contrato_meses': 'Tempo de Contrato (meses)',
            var: titulos[i]
        }
    )
    show_legend = (i == 1)
    for trace in scatter.data:
        trace.showlegend = show_legend
        fig.add_trace(trace, row=1, col=i+1)

# Layout final
fig.update_layout(
    title='Relação entre Tempo de Contrato e Variáveis Financeiras',
    showlegend=True,
    legend_title='Evasão',
    height=500,
    xaxis2=dict(
        title='Tempo de Contrato (meses)',
        title_standoff=40
    ),
    yaxis_title='Variáveis Financeiras',
    template='plotly_white'
)

fig.show()

### **Avaliando valores categóricos junto a evasão de clientes**

### Evasão por gênero

In [ ]:
taxa_por_genero = df_flat.groupby('genero')['evasao'].mean()

plt.figure(figsize=(6,4))
plt.bar(taxa_por_genero.index, taxa_por_genero.values, color=[AQUA_2, VERMELHO_1])
plt.title('Taxa de evasão por Gênero', fontsize=14, color=CINZA_5, loc='left')
plt.ylabel('Proporção de Evasão', fontsize=12, color=CINZA_5)
plt.ylim(0, taxa_por_genero.max() + 0.1)
for i, v in enumerate(taxa_por_genero.values):
    plt.text(i, v + 0.01, f"{v:.2f}", ha='center', color=CINZA_5)
ax = plt.gca()
ax.spines[['top','right']].set_visible(False)
plt.show()

## Evasão por tipo de contrato

In [ ]:
taxa_por_contrato = df_flat.groupby('tipo_contrato')['evasao'].mean().sort_values()

plt.figure(figsize=(10,6))
plt.bar(taxa_por_contrato.index, taxa_por_contrato.values, color=AZUL_1)
plt.title('Taxa de evasão por Tipo de Contrato', fontsize=14, color=CINZA_5, loc='left')
plt.ylabel('Proporção de Evasão', fontsize=12, color=CINZA_5)
plt.xticks(rotation=30)
for i, v in enumerate(taxa_por_contrato.values):
    plt.text(i, v + 0.01, f"{v:.2f}", ha='center')
ax = plt.gca()
ax.spines[['top','right']].set_visible(False)
plt.tight_layout()
plt.show()

## Evasão por forma de pagamento

In [ ]:
taxa_por_pagamento = df_flat.groupby('forma_pagamento')['evasao'].mean().sort_values()

plt.figure(figsize=(12,6))
plt.barh(taxa_por_pagamento.index, taxa_por_pagamento.values, color=[AQUA_1, AQUA_1, AQUA_1, VERMELHO_1])
plt.title('Taxa de evasão por Forma de Pagamento', fontsize=14, color=CINZA_5, loc ='left')
plt.xlabel('Proporção de Evasão', fontsize=12, color=CINZA_5)
for i, v in enumerate(taxa_por_pagamento.values):
    plt.text(v + 0.005, i, f"{v:.2f}", va='center')
ax = plt.gca()
ax.spines[['top','right']].set_visible(False)
plt.tight_layout()
plt.show()

## **Distribuição de evasão por tipo de contrato**

In [ ]:
ct = pd.crosstab(df_flat['tipo_contrato'], df_flat['evasao'], normalize='index')

ct.plot.bar(stacked=True, color=[AQUA_2, VERMELHO_1], figsize=(13,9))
plt.title('Distribuição de Evasão (%) por Tipo de Contrato', fontsize=14, color=CINZA_5, loc='left')
plt.ylabel('Proporção', fontsize=12)
plt.tick_params(axis='x', labelrotation = 45)
plt.gca().spines[['top','right']].set_visible(False)
plt.legend(['Ficaram','Perderam'], title='Status', loc='upper right')

plt.show()


## **Evasão por tipo de internet**

In [ ]:
taxa_por_internet = df_flat.groupby('tipo_internet')['evasao'].mean().sort_values()

plt.figure(figsize=(10,6))
plt.bar(taxa_por_internet.index, taxa_por_internet.values, color=AQUA_3)
plt.title('Taxa de evasão por Tipo de Internet', fontsize=14, color=CINZA_5, loc='left')
plt.ylabel('Proporção de Evasão', fontsize=12, color=CINZA_5)
plt.xticks(rotation=30)
for i, v in enumerate(taxa_por_internet.values):
    plt.text(i, v + 0.01, f"{v:.2f}", ha='center')
ax = plt.gca()
ax.spines[['top','right']].set_visible(False)
plt.tight_layout()
plt.show()

## **Evasão por fatura**

In [ ]:
taxa_por_fatura = df_flat.groupby('fatura_online')['evasao'].mean().sort_index()

plt.figure(figsize=(10,8))
plt.bar(taxa_por_fatura.index.map({0:'Papel',1:'Online'}), taxa_por_fatura.values, color=AZUL_3)
plt.title('Taxa de evasão por Fatura Online', fontsize=14, color=CINZA_5, loc='left')
plt.ylabel('Proporção de Evasão', fontsize=12, color=CINZA_5)
for i, v in enumerate(taxa_por_fatura.values):
    plt.text(i, v + 0.01, f"{v:.2f}", ha='center')
ax = plt.gca()
ax.spines[['top','right']].set_visible(False)
plt.tight_layout()
plt.show()

## **Evasão por serviços**

In [ ]:
service_columns = [
    'servico_telefonico', 'multiplas_linhas', 'seguranca_online',
    'backup_online', 'protecao_dispositivo', 'suporte_tecnico',
    'streaming_tv', 'streaming_filmes'
]


taxa_por_servico = df_flat[service_columns + ['evasao']].groupby(service_columns).mean()['evasao']

taxa_ativos = {}
for serv in service_columns:
    taxa_ativos[serv] = df_flat.loc[df_flat[serv] == 1, 'evasao'].mean()

#Plot
fig, ax = plt.subplots(figsize=(10, 6))
serv_names = [s.replace('_', ' ').title() for s in service_columns]
valores = [taxa_ativos[s] for s in service_columns]

ax.bar(serv_names, valores, color=AQUA_3)
ax.set_title('Taxa de Evasão por Serviço Ativo', fontsize=14, color=CINZA_5, loc='left')
ax.set_ylabel('Proporção de Evasão', fontsize=12, color=CINZA_5)


ax.set_xticklabels(serv_names, rotation=45, ha='right', color=CINZA_5)
ax.set_xticks(serv_names)


for i, v in enumerate(valores):
    ax.text(i, v + 0.01, f"{v:.2f}", ha='center', color=CINZA_5)

for spine in ['top', 'right']:
    ax.spines[spine].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
df_clean[service_columns].sum()

# ATIVIDADE EXTRA CORRELADAÇÃO DE DADOS

In [ ]:
service_columns = [
    'servico_telefonico', 'multiplas_linhas', 'seguranca_online',
    'backup_online', 'protecao_dispositivo', 'suporte_tecnico',
    'streaming_tv', 'streaming_filmes'
]

num_cols = ['evasao', 'tempo_contrato_meses', 'gastos_mensais',
            'gastos_totais', 'contas_diarias'] + service_columns

corr_df = df_flat[num_cols].corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr_df, annot=True, fmt=".2f", cmap="Blues", vmin=-1, vmax=1,
            cbar_kws={'label': 'Coeficiente de Correlação'})
plt.title("Matriz de Correlação com Variáveis de Serviços Individuais", color=CINZA_5)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.boxplot(x='evasao', y='contas_diarias', data=df_flat,
            palette=[AQUA_2, VERMELHO_1])
plt.title("Contas Diárias x Evasão", color=CINZA_5)
plt.xlabel("Evasão (0 = Ficou, 1 = Saiu)")
plt.ylabel("Contas Diárias")
plt.show()

In [ ]:
plt.figure(figsize=(6,4))
sns.violinplot(x='evasao', y='contas_diarias', data=df_flat,
               palette=[AQUA_2, VERMELHO_1])
plt.title("Distribuição de Contas Diárias por Evasão", color=CINZA_5)
plt.xlabel("Evasão (0 = Ficou, 1 = Saiu)")
plt.ylabel("Contas Diárias")
plt.show()

# Relatório Final

## **Projeto Telecom X: Análise de Evasão de Clientes (Churn)**

---

### 1. **Introdução**

A evasão de clientes (churn) é sempre um grande desafio para operadoras de telecomunicações. Nesta análise, recebemos a demanda de exploramos dados de mais de 7 000 clientes da Telecom X para entender quem está cancelando sua conta, quando e por quê. Nosso objetivo é identificar padrões de comportamento e fatores de risco que permitam à liderança tomar decisões estratégicas de retenção, reduzindo a perda de receita e fortalecendo o relacionamento com o cliente.

---

### 2. **Limpeza e Tratamento de Dados**

- Importação e flattening

    - Lemos o JSON bruto via pd.read_json e “achamos” todas as colunas aninhadas com pd.json_normalize.

- Renomeação de colunas

    - Padronizamos nomes para snake_case e traduzimos para o português (por exemplo, customerid → id_cliente, Churn → evasao, InternetService → tipo_internet etc.).

- Conversão de tipos e tratamento de ausentes

    - Transformamos gastos_mensais, gastos_totais e tempo_contrato_meses em numéricos, corrigimos strings inválidas, como NaN, e preenchemos zeros onde fazia sentido.

    - Identificamos e removemos duplicatas e padronizamos respostas como “No internet service” → “No”.

- Padronização de valores Binários para inteiros

    - Substituímos todos os “Yes”/“No” por 1/0 nas colunas de serviços e preferências, garantindo int64.

- Criação de novas métricas

    - contas_diarias = gastos_totais / tempo_contrato_meses

    - num_services = soma dos 8 serviços extras (telefone, múltiplas linhas, segurança online, backup, proteção de dispositivo, suporte técnico, streaming de TV e filmes).

---

### 3. **Análise Exploratória de Dados**

    3.1 Perfil Geral

- Taxa de churn: 26,6% dos clientes cancelaram.

- Tempo de contrato: mediana de 29 meses; 25% dos clientes têm até 9 meses de vínculo.

- Gastos mensais: média de R\$ 64,80; contas diárias média de R\$ 2,15.

      3.2 Principais análises bivariadas

- Contrato vs. Evasão
    - Month-to-month: 42,7% de churn

    - One-year: 11,3%

    - Two-year: 2,9%

- Tempo de Contrato
    - Boxplot e histogramas mostram que clientes que realizam a evasão de contranto concentram-se em vínculos muito curtos (majoritariamente abaixo de 12 meses).

- Gastos vs. Evasão
    - Scatter plots de tempo × gastos mensais, totais e contas diárias indicam que churners têm maior concentração em baixos valores diários e mensais.

- Serviços Extras
    - Taxas de churn entre clientes em cada serviço:

    - Suporte Técnico: 42,2%

    - Segurança Online: 41,8%

    - Streaming TV e Filmes: ~40%

- Demografia e Pagamentos
    - Idosos: 41,7% churn

    - Cheque Eletrônico: 45,3% churn

    - Fatura sem papel: 33,8% churn

    - Sem parceiro ou sem dependentes: ~32% churn

          3.3 Correlação de Variáveis Numéricas

- Matriz de correlação mostra relações moderadas entre churn e:

    - Tempo de contrato

    - Serviços

    - Contas diárias

---

### 4. **Conclusões e Insights**

- Risco elevado nos primeiros meses: quase metade dos cancelamentos ocorre dentro dos 12 meses iniciais.

- Contratos longos reduzem churn: planos de 1–2 anos apresentam evasão muito menor que o plano mensal.

- Pacotes de serviços fidelizam: clientes com mais serviços extras têm menor churn.

- Qualidade de Fibra Óptica: apesar da alta demanda, apresenta churn alto — precisa de investigação para identificar se o serviço está entregando uma boa qualidade para os clientes.  

- Método de pagamento e perfil: cheque eletrônico e clientes idosos estão em maior risco.

---

### 5. **Recomendações**

- Programa de acompanhamento nos primeiros 12 meses, com ofertas de trial de serviços extras.

- Incentivos Contratuais para fazer com que o clientes contrate periodos maiores

- Descontos e benefícios para migração de Month-to-month para contratos anuais logo no início.

- Otimização de Serviços

- Auditoria de performance da Fibra Óptica e buscar escutar os clientes com formulários de satisfação ou ainda registros com o motivo do cancelamento.

- Promoção de pacotes combinados (suporte + segurança + backup) com valor agregado. No geral, clientes que contratam mais serviços de uma mesma empresa, tendem a não querer cancelar o vinculo, pois é mais complicado migrar para outras plataformas.

- Aprimoramento de Pagamentos

- Revisão da experiência de cheque eletrônico e incentivos ao uso de métodos automáticos.

- Ações específicas para idosos, solteiros e sem dependentes, com canais de suporte dedicados. Buscando inserir o idoso dentro dos programas de beneficio da empresa e assim demonstrando que a empresa preza uma serviço atensioso aos seus clientes.

- Clientes solteiros e sem dependentes é a parte mais difícel de evitar evasão, pois provavelmente o perfil desses clientes é baseado em pessoas jovens, sem moradia fixa, que estão construindo a vida, ou seja, muitas mudanças acontecendo que podem levar a evasão de contrato.  

---

### **Próximos Passos**

- Desenvolver modelo de ML para entender melhor o comportamento dos dados.

- Coletar dados de linha do tempo (datas de contrato e churn), histórico de atendimento e NPS.

- Monitorar indicadores em dashboards semanais para avaliar o impacto das ações.Buscar realizar pesquisas de satisfação e registras sempre os motivos do cancelamento, assim como, qual serviço foi cancelado que gerou a evasão desse cliente.

- Com essas iniciativas, a Telecom X terá ideias sólidas para reduzir seu churn com base nos dados, dessa forma, elevando a satisfação dos clientes e maximizando o valor de vida útil de cada contrato.